In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

tickers = ['PANW', 'SHOP', 'META', 'TSM', 'NET', 'DELL', 'ON', 'CRM', 'SONY', 'CRWD', 'AMAT']

def get_data(stock):
    #get historical stock data 
    YEARS = 20
    #try catch for years in case of error 
    start = (pd.Timestamp.now() - pd.DateOffset(years=YEARS)).strftime('%Y-%m-%d')
    END = pd.Timestamp.now().strftime('%Y-%m-%d')
    stock_data = yf.download(stock, start=start, end=END)
    
    # Calculate daily returns
    stock_data['Daily Return'] = stock_data['Close'].pct_change()
    
    # Separate gains and losses
    stock_data['Gain'] = stock_data['Daily Return'].apply(lambda x: x if x > 0 else 0)
    stock_data['Loss'] = stock_data['Daily Return'].apply(lambda x: -x if x < 0 else 0)
    
    # Calculate average gain and loss
    window = 14
    stock_data['Avg Gain'] = stock_data['Gain'].rolling(window=window).mean()
    stock_data['Avg Loss'] = stock_data['Loss'].rolling(window=window).mean()
    
    # Calculate RS and RSI
    stock_data['RS'] = stock_data['Avg Gain'] / stock_data['Avg Loss']
    stock_data['RSI'] = 100 - (100 / (1 + stock_data['RS']))
    # Define the ticker symbol, start date, and end date
    ticker_symbol = 'AAPL'  # Example: Apple Inc.
    start_date = '2023-01-01'
    end_date = '2023-12-31'

# Fetch historical data from Yahoo Finance
    data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Calculate the 26-day moving average of the closing prices
    data['26_Day_MA'] = data['Close'].rolling(window=26).mean()

# Display the data with the moving average
    print(data[['Close', '26_Day_MA']])

    # Return the final DataFrame
    return stock_data[['Close', 'RSI']]

print(get_data('AAPL'))



[*********************100%%**********************]  1 of 1 completed
                 Close        RSI
Date                             
2004-03-01    0.428929        NaN
2004-03-02    0.425179        NaN
2004-03-03    0.427143        NaN
2004-03-04    0.449286        NaN
2004-03-05    0.477500        NaN
...                ...        ...
2024-02-22  184.369995  42.550837
2024-02-23  182.520004  40.416768
2024-02-26  181.160004  30.541082
2024-02-27  182.630005  30.139032
2024-02-28  181.419998  27.621140

[5034 rows x 2 columns]
